The first part of the tutorial is inspired by the single morphometrics analysis proposed in Guijarro et al., 2024 [Github](https://github.com/VILLOUTREIXLab/sc-morphometrics-SHF), [paper](https://www.nature.com/articles/s41467-024-53612-8)

# Single-cell morphometrics reveals T-box dependent patterns of epithelial tension in the Second Heart Field

### Goal: 
Understand the patterning of cardiac progenitors.The embryonic heart grows by the progressive addition of progenitor cells located on an epithelial layer known as the Dorsal pericardial wall (DPW).
T-box transcription genes (Tbx1 and Tbx5) play a key role in regulating the addition of the cells to the poles of the heart. However, the mechanisms by which the cells contribute to each pole remain unclear.

![E8 5 to E9 5 transition horizontal graph v3](https://github.com/user-attachments/assets/ac37731b-2141-4eb7-97a4-b89efddd72b4)
To investigate this, we have employed dimentionality reduction and unsupervised clustering algorythms to map the cells according to their morphological features.

### Data:
- The data used for the sc-morphometricanalysis is stored in the Data folder. The code for quantification of sc-morphometrics clusters is in Quantification clusters - final.ipnyb, the code for the clusters obtained using absolute angle values is in Quantification clusters_abs degrees.ipnyb.
- The data used for RNAscope quantification is stored in Data_RNAscope folder. The code for quantification is RNAscope - mRNA quantification - final.ipnyb
- The images from wild-type, mutants and BMS treatment passed through Tissue Analyser for their segmentation and used in the study are in the IMAGES folder.

### Pipeline:
![figure1](https://github.com/user-attachments/assets/edccac6e-3534-4fe0-8c57-3da490ac7e71)

1) Data acquisition: Whole mount immunofluorescence on the Dorsal pericardial wall (the cardiac epithelium) using phalloidin to label the cell membranes and other proteins of interest.
2) Image segmentation using [Tissue Analyser](https://github.com/baigouy/tissue_analyzer) (TA) as described by Aigouy et al.2016, [Force inference](https://data.mendeley.com/datasets/78ng4tmj75/4) (Kong et al. 2019) and application of [Dproj](https://gitlab.pasteur.fr/iah-public/DeProj) (Herbert et al. 2021).
3) Store,clean and merge the data

In this tutorial, we will analyze the data using joint-spatial PCA 

In [ ]:
import pandas as pd
import numpy as np
import copy
import os
from scipy.io import savemat

# data/input folder contains
# 1)all the adjacency matrices as dataframes using the cells names cell_id as indices and columns names
# the adjacency matrices should be named 'Ae'+str(e_id) for each e_id
# 2)the dataframe of all centered and scaled features, called 'df_features_scaled.pkl'
# with a column 'embryo' for e_id of the embryos 


# the scaled features can be found in the following data frame
df_features_scaled=pd.read_pickle('data/scmorphometrics/df_features_scaled.pkl')
e_ids=[1,2,3,4,5,6]
n_features=8

# the positions of individual cells can be found the following data frame
df_abs=pd.read_pickle('data/scmorphometrics/df_positions_labelled_def.pkl')


merged_df = pd.merge(df_features_scaled, df_abs )

# visualize the head
# merged_df.head


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import show 
import seaborn as sns 

# question 1 - visualize each of the feature in the tissue space for a given embryo



In [ ]:
# question 2 - visualize the eigenvalues of the PCA of the scaled features


In [ ]:
# from the position of the cells, it is possible to compute the adjacency matrix with k-Nearest Neighbor network 
# or Delaunay Triangulation
# here we have the adjacency matrix as an input in the folder data/scmorphometrics
# as dataframes using the cells names cell_id as indices and columns names
# the adjacency matrices are named 'Ae'+str(e_id) for each e_id (embryo id) 

# question 3 - compute the degree distribution of a given embryo

# L is the row normalized adjacency matrix

# question 4 (optional) - compute the Moran's Index of the Principal Components using the formulation X^t L X / X^t X


In [ ]:
# question 5 - compute sPCA as the eigenvectors of the score matrix 1/2nX^t(L+L^t)X
# compare the coefficient with the ones obtained with simple PCA


In [ ]:
# question 6 - perform 3 class clustering on the first component of sPCA and visualize the classes in the tissue space


# Spatial transcriptomics


The second part of the tutorial is dedicated to the spatial analysis of spatial transcriptomics data using [jsPCA](https://github.com/VILLOUTREIXLab/jsPCA/tree/main)

We will base our analysis on the [DLPFC 10x Visium dataset](http://sdmbench.drai.cn), based on slice 151673 (file number 9 in the database).

In [ ]:
import warnings
import os
import glob
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (normalized_mutual_info_score, 
                             adjusted_rand_score, 
                             accuracy_score,
                             homogeneity_score,
                             completeness_score,
                             silhouette_score)
from scipy.optimize import linear_sum_assignment
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
import tracemalloc
import time


# Suppress warnings
warnings.filterwarnings("ignore")

# Define the folder path where all the .h5ad files are stored
folder_path = "data/visium/"

# # Get a list of all .h5ad files in the folder
# file_paths = glob.glob(os.path.join(folder_path, "*.h5ad"))

# Example: Read the specific .h5ad file (for example, "151673.h5ad")
file_to_read = "151673.h5ad"
adata = ad.read_h5ad(os.path.join(folder_path, file_to_read))
new_adata = ad.read_h5ad(os.path.join(folder_path, file_to_read))

# Display the data
print(adata)

# Normalization
sc.pp.filter_genes(adata,min_cells=20)
sc.experimental.pp.normalize_pearson_residuals(adata)
sc.pp.scale(adata)

# Get the count of unique labels in 'ground_truth', ignoring NaN values
num_labels = adata.obs['ground_truth'].nunique()
print(f"Number of unique labels in 'ground_truth' (excluding NaN) for {file_to_read}: {num_labels}")

# visualize the ground truth data 
plt.rcParams["figure.figsize"] = (3, 3)
sc.pl.spatial(adata, img_key="hires", color=["Region"], spot_size=100)

In [ ]:
# question 1 : Compute the adjacency matrix with k-NN



In [ ]:
# question 2 : Compute the jsPCA monoslice and its eignevalues and eigenvectors - 1/2n X^t (L + L^t) X


In [ ]:
# question 3 : Project the data on the first 10 eigenvectors of jsPCA and classify them using Gaussian Mixture Model


In [ ]:
# question 4 : Compare the prediction to the ground truth


In [ ]:
# question 5: Compute the ARI (Adjusted Rand Index) and NMI (Normalized Mutual Information)


In [ ]:
# question 6: Compare the results with a clustering performed on simple PCA (without taking into account the space)


In [ ]:
# question 7 (optional): compare the results with other spatially aware clustering methods
